In [2]:
import numpy as np
import dask , dask.distributed
import xarray as xr
import matplotlib.pyplot as plt
import cmocean.cm as cmo
import warnings
warnings.simplefilter('ignore')
import dask_jobqueue
from astropy.convolution import Box2DKernel, convolve
#import xskillscore as xs
from filter import filtering
from ice import create_icemask, ice_filter
from calculations import concat_data, anomalies, corr, corr_all, seas_corr, concat_jp

# create ice mask, mask out ice

In [ ]:
##icemasks and save, don't run again, find all regions without any ==0 ice
icemask_ctrl=create_icemask(ds_ctrl)
icemask_cc=create_icemask(ds_cc)
icemask_ctrl.to_netcdf('/gxfs_work/geomar/smomw577/mesoscale_eddies/BOX_filtered/0181-0190/icemask_control.nc')
icemask_cc.to_netcdf('/gxfs_work/geomar/smomw577/mesoscale_eddies/BOX_filtered/0181-0190/icemask_1PctTo2X.nc')

In [55]:
### load this file instead
icemask_ctrl=xr.open_dataset('/gxfs_work/geomar/smomw577/mesoscale_eddies/BOX_filtered/0181-0190/icemask_control.nc')
icemask_cc=xr.open_dataset('/gxfs_work/geomar/smomw577/mesoscale_eddies/BOX_filtered/0181-0190/icemask_1PctTo2X.nc')

In [9]:
ds_ctrl=xr.open_dataset('/gxfs_work/geomar/smomw577/mesoscale_eddies/MOM5_concat/0181-0190/MOM5_control_monthly_0181-0190_all.nc')
ds_cc=xr.open_dataset('/gxfs_work/geomar/smomw577/mesoscale_eddies/MOM5_concat/0181-0190/MOM5_1PctTo2X_monthly_0181-0190_all.nc')

In [57]:
ice_filter(ds_ctrl, icemask_ctrl.EXT,'/gxfs_work/geomar/smomw577/mesoscale_eddies/MOM5_concat/0181-0190/MOM5_control_monthly_noice_0181-0190_all.nc' )
ice_filter(ds_cc, icemask_cc.EXT,'/gxfs_work/geomar/smomw577/mesoscale_eddies/MOM5_concat/0181-0190/MOM5_1PctTo2X_monthly_noice_0181-0190_all.nc' )

# take mean and stdev

In [7]:
ds=xr.open_dataset('/gxfs_work/geomar/smomw577/mesoscale_eddies/MOM5_concat/0181-0190/MOM5_control_monthly_0181-0190_noice_all.nc')
ds_cc=xr.open_dataset('/gxfs_work/geomar/smomw577/mesoscale_eddies/MOM5_concat/0181-0190/MOM5_1PctTo2X_monthly_0181-0190_noice_all.nc')

In [3]:
mean=ds.mean(dim='time')
stdev=ds.std(dim='time')
mean_cc=ds_cc.mean(dim='time')
stdev_cc=ds_cc.std(dim='time')

In [4]:
for item in list(mean.keys()):
    mean=mean.rename({item:item+'_mean'})
    stdev=stdev.rename({item:item+'_std'})
    mean_cc=mean_cc.rename({item:item+'_mean'})
    stdev_cc=stdev_cc.rename({item:item+'_std'})


In [5]:
ds_mean=xr.merge([mean, stdev])
dscc_mean=xr.merge([mean_cc, stdev_cc])
ds_mean.to_netcdf('/gxfs_work/geomar/smomw577/mesoscale_eddies/MOM5_concat/0181-0190/mean_std_control_noice.nc')
dscc_mean.to_netcdf('/gxfs_work/geomar/smomw577/mesoscale_eddies/MOM5_concat/0181-0190/mean_std_1PctTo2X_noice.nc')

# filtering all data 181-190

In [2]:
ds=xr.open_dataset('/gxfs_work/geomar/smomw577/mesoscale_eddies/MOM5_concat/0181-0190/MOM5_control_monthly_0181-0190_noice_all.nc')
dscc=xr.open_dataset('/gxfs_work/geomar/smomw577/mesoscale_eddies/MOM5_concat/0181-0190/MOM5_1PctTo2X_monthly_0181-0190_noice_all.nc')

save='/gxfs_work/geomar/smomw577/mesoscale_eddies/BOX_filtered/0181-0190/3x3box_mean_monthly_0181-0190_noice.nc'
savecc='/gxfs_work/geomar/smomw577/mesoscale_eddies/BOX_filtered/0181-0190/3x3box_mean_monthly_1PctTo2X_0181-0190_noice.nc'

In [13]:
ds_ano_cc=filtering(dscc)
ds_ano_cc.to_netcdf(savecc)

ds_ano=filtering(ds)
ds_ano.to_netcdf(save)

# save anomalies in an extra file

In [3]:
ds_ctrl= '/gxfs_work/geomar/smomw577/mesoscale_eddies/MOM5_concat/0181-0190/MOM5_control_monthly_0181-0190_noice_all.nc'
ds_filtered_ctrl= '/gxfs_work/geomar/smomw577/mesoscale_eddies/BOX_filtered/0181-0190/3x3box_mean_monthly_0181-0190_noice.nc'

ds_cc='/gxfs_work/geomar/smomw577/mesoscale_eddies/MOM5_concat/0181-0190/MOM5_1PctTo2X_monthly_0181-0190_noice_all.nc'
ds_filtered_cc='/gxfs_work/geomar/smomw577/mesoscale_eddies/BOX_filtered/0181-0190/3x3box_mean_monthly_1PctTo2X_0181-0190_noice.nc'

save_ctrl='/gxfs_work/geomar/smomw577/mesoscale_eddies/BOX_filtered/0181-0190/3x3box_mean_anomaly_monthly_0181-0190.nc'
save_cc='/gxfs_work/geomar/smomw577/mesoscale_eddies/BOX_filtered/0181-0190/3x3box_mean_anomaly_monthly_1PctTo2X_0181-0190.nc'

anomalies(ds_ctrl, ds_filtered_ctrl, save_ctrl)
anomalies(ds_cc, ds_filtered_cc, save_cc)

<xarray.Dataset>
Dimensions:    (time: 120, xt_ocean: 3600, yt_ocean: 2700)
Coordinates:
  * time       (time) object 0181-01-16 12:00:00 ... 0190-12-16 12:00:00
  * xt_ocean   (xt_ocean) float64 -279.9 -279.8 -279.7 ... 79.75 79.85 79.95
  * yt_ocean   (yt_ocean) float64 -81.11 -81.07 -81.02 ... 89.89 89.94 89.98
    st_ocean   float64 5.034
Data variables:
    SST        (time, yt_ocean, xt_ocean) float32 nan nan nan ... nan nan nan
    SSH        (time, yt_ocean, xt_ocean) float32 nan nan nan ... nan nan nan
    dic_stf    (time, yt_ocean, xt_ocean) float32 nan nan nan ... nan nan nan
    o2_stf     (time, yt_ocean, xt_ocean) float32 nan nan nan ... nan nan nan
    sens_heat  (time, yt_ocean, xt_ocean) float32 nan nan nan ... nan nan nan
    evap_heat  (time, yt_ocean, xt_ocean) float32 nan nan nan ... nan nan nan
    mld        (time, yt_ocean, xt_ocean) float32 nan nan nan ... nan nan nan
    jp_all     (time, yt_ocean, xt_ocean) float64 nan nan nan ... nan nan nan

In [8]:
suffix = "_1PctTo2X"
rename_dict = {var: var + suffix for var in ds_cc.data_vars}
ds_renamed = ds_cc.rename_vars(rename_dict)
ds_renamed.to_netcdf('/gxfs_work/geomar/smomw577/mesoscale_eddies/MOM5_concat/0181-0190/MOM5_1PctTo2X_monthly_0181-0190_noice_all_new.nc')

In [ ]:
ds_raw= '/gxfs_work/geomar/smomw577/mesoscale_eddies/MOM5_concat/0181-0190/MOM5_monthly_0181-0190_noice_all.nc'
ds_filtered= '/gxfs_work/geomar/smomw577/mesoscale_eddies/BOX_filtered/0181-0190/3x3box_median_monthly_0181-0190_noice_all.nc'

save='/gxfs_work/geomar/smomw577/mesoscale_eddies/BOX_filtered/0181-0190/3x3box_median_anomaly_monthly_0181-0190_all.nc'

anomalies(ds_raw, ds_filtered, save)

# correlations all time steps with SST/SSH

In [4]:
pathf='/gxfs_work/geomar/smomw577/mesoscale_eddies/BOX_filtered/0181-0190/'

ctrl=pathf+'3x3box_median_anomaly_monthly_0181-0190_all.nc'
save_ctrl_noice=pathf+'3x3box_median_corr_monthly_0181-0190_noice_all.nc'

#cc=pathf+'3x3box_mean_anomaly_monthly_1PctTo2X_0181-0190.nc'
#save_cc_noice=pathf+'3x3box_mean_corr_monthly_1PctTo2X_0181-0190_noice.nc'

In [ ]:
corr_all(ctrl, save_ctrl_noice)

In [7]:
#corr_all(cc, save_cc_noice)

# seasonal correlations

In [8]:
pathf='/gxfs_work/geomar/smomw577/mesoscale_eddies/BOX_filtered/0181-0190/'
#control
ctrl=pathf+'3x3box_mean_anomaly_monthly_0181-0190.nc'
save_ctrl_noice=pathf+'3x3box_mean_corr_seasonally_0181-0190_noice.nc'
#climate change
cc=pathf+'3x3box_mean_anomaly_monthly_1PctTo2X_0181-0190.nc'
save_cc_noice=pathf+'3x3box_mean_corr_seasonally_1PctTo2X_0181-0190_noice.nc'

seas_corr(ctrl, save_ctrl_noice)
seas_corr(cc, save_cc_noice)

# correlation dic and o2

In [5]:
#dsa=xr.open_dataset(pathf+'3x3box_mean_anomaly_monthly_0181-0190.nc')
dsa=xr.open_dataset(pathf+'3x3box_median_anomaly_monthly_0181-0190_all.nc')
corrssh=xr.corr(dsa.o2_stf, dsa['dic_stf'], dim='time')
corrssh2=xr.corr(dsa.o2_stf_1PctTo2X, dsa['dic_stf_1PctTo2X'], dim='time')
corrssh.name='corr_o2_stf_'+'dic_stf'
corrssh2.name='corr_o2_stf_'+'dic_stf_1PctTo2X'

In [6]:
savefile=[corrssh, corrssh2]
#xr.merge(savefile).to_netcdf(pathf+'3x3box_mean_corr_monthly_co2-o2_0181-0190_noice.nc')
xr.merge(savefile).to_netcdf(pathf+'3x3box_median_corr_monthly_co2-o2_0181-0190_noice_all.nc')

# correlation dic/o2 and jp_uptake

In [ ]:
pathf='/gxfs_work/geomar/smomw577/mesoscale_eddies/BOX_filtered/0181-0190/'

#dsa=xr.open_dataset(pathf+'3x3box_mean_anomaly_monthly_0181-0190_all.nc')
dsa=xr.open_dataset(pathf+'3x3box_median_anomaly_monthly_0181-0190_all.nc')

corr1=xr.corr(dsa.jp_all, dsa['dic_stf'], dim='time')
corr1.name='corr_jp_'+'dic_stf_ctrl'

corr2=xr.corr(dsa.jp_all, dsa['o2_stf'], dim='time')
corr2.name='corr_jp_'+'o2_stf_ctrl'

#dsa2=xr.open_dataset(pathf+'3x3box_mean_anomaly_monthly_1PctTo2X_0181-0190.nc')
corr3=xr.corr(dsa.jp_all_1PctTo2X, dsa['dic_stf_1PctTo2X'], dim='time')
corr3.name='corr_jp_'+'dic_stf_1PctTo2X'

corr4=xr.corr(dsa.jp_all_1PctTo2X, dsa['o2_stf_1PctTo2X'], dim='time')
corr4.name='corr_jp_'+'o2_stf_1PctTo2X'

savefile=[corr1, corr2, corr3, corr4]
xr.merge(savefile).to_netcdf(pathf+'3x3box_median_corr_monthly_jp_0181-0190_noice_all.nc')

# correlation dic/o2 and mld

In [ ]:
pathf='/gxfs_work/geomar/smomw577/mesoscale_eddies/BOX_filtered/0181-0190/'

#dsa=xr.open_dataset(pathf+'3x3box_mean_anomaly_monthly_0181-0190.nc')
dsa=xr.open_dataset(pathf+'3x3box_median_anomaly_monthly_0181-0190_all.nc')

corr1=xr.corr(dsa.mld, dsa['dic_stf'], dim='time')
corr1.name='corr_mld_'+'dic_stf_ctrl'

corr2=xr.corr(dsa.mld, dsa['o2_stf'], dim='time')
corr2.name='corr_mld_'+'o2_stf_ctrl'

#dsa2=xr.open_dataset(pathf+'3x3box_mean_anomaly_monthly_1PctTo2X_0181-0190.nc')
corr3=xr.corr(dsa.mld_1PctTo2X, dsa['dic_stf_1PctTo2X'], dim='time')
corr3.name='corr_mld_'+'dic_stf_1PctTo2X'

corr4=xr.corr(dsa.mld_1PctTo2X, dsa['o2_stf_1PctTo2X'], dim='time')
corr4.name='corr_mld_'+'o2_stf_1PctTo2X'

In [ ]:
savefile=[corr1, corr2, corr3, corr4]
#xr.merge(savefile).to_netcdf(pathf+'3x3box_mean_corr_monthly_mld_0181-0190_noice.nc')
xr.merge(savefile).to_netcdf(pathf+'3x3box_median_corr_monthly_mld_0181-0190_noice_all.nc')